In [1]:
import os
import tensorflow as tf
import tensorflow_model_analysis as tfma
from tfx.components import CsvExampleGen, StatisticsGen, SchemaGen, ExampleValidator, Transform, Trainer, Tuner, Evaluator, Pusher
from tfx.proto import example_gen_pb2, trainer_pb2, transform_pb2, pusher_pb2
from tfx.orchestration.experimental.interactive.interactive_context import InteractiveContext
from tfx.dsl.components.common.resolver import Resolver
from tfx.dsl.input_resolution.strategies.latest_blessed_model_strategy import LatestBlessedModelStrategy
from tfx.types import Channel
from tfx.types.standard_artifacts import Model, ModelBlessing

2024-11-05 05:51:10.910354: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-11-05 05:51:10.953340: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
PIPELINE_NAME = 'churn-pipeline'
SCHEMA_PIPELINE_NAME = 'churn-tfdv-schema'

PIPELINE_ROOT = os.path.join('pipelines', PIPELINE_NAME)
METADATA_PATH = os.path.join('metadata', PIPELINE_NAME, 'metadata.db')
SERVING_MODEL_DIR = os.path.join('serving_model', PIPELINE_NAME)

DATA_ROOT = 'data'

In [3]:
context = InteractiveContext(pipeline_root=PIPELINE_ROOT)

In [4]:
input = example_gen_pb2.Input(splits=[
    example_gen_pb2.Input.Split(name='train', pattern='train*'),
    example_gen_pb2.Input.Split(name='eval', pattern='val*')
])
example_gen = CsvExampleGen(input_base=DATA_ROOT, input_config=input)

context.run(example_gen)

ExecutionResult(
    component_id: CsvExampleGen
    execution_id: 1
    outputs:
        examples: OutputChannel(artifact_type=Examples, producer_component_id=CsvExampleGen, output_key=examples, additional_properties={}, additional_custom_properties={}, _input_trigger=None)

In [5]:
statistics_gen = StatisticsGen(
    examples=example_gen.outputs['examples']
)

context.run(statistics_gen)

ExecutionResult(
    component_id: StatisticsGen
    execution_id: 2
    outputs:
        statistics: OutputChannel(artifact_type=ExampleStatistics, producer_component_id=StatisticsGen, output_key=statistics, additional_properties={}, additional_custom_properties={}, _input_trigger=None)

In [6]:
context.show(statistics_gen.outputs['statistics'])

In [7]:
schema_gen = SchemaGen(
    statistics=statistics_gen.outputs['statistics']
)

context.run(schema_gen)

ExecutionResult(
    component_id: SchemaGen
    execution_id: 3
    outputs:
        schema: OutputChannel(artifact_type=Schema, producer_component_id=SchemaGen, output_key=schema, additional_properties={}, additional_custom_properties={}, _input_trigger=None)

In [8]:
context.show(schema_gen.outputs['schema'])

,Type,Presence,Valency,Domain
Feature name,,,,
'Age',INT,required,,-
'Churn',INT,required,,-
'Contract Length',STRING,required,,'Contract Length'
'CustomerID',INT,required,,-
'Gender',STRING,required,,'Gender'
'Last Interaction',INT,required,,-
'Payment Delay',INT,required,,-
'Subscription Type',STRING,required,,'Subscription Type'
'Support Calls',INT,required,,-


,Values
Domain,
'Contract Length',"'Annual', 'Monthly', 'Quarterly'"
'Gender',"'Female', 'Male'"
'Subscription Type',"'Basic', 'Premium', 'Standard'"


In [9]:
example_validator = ExampleValidator(
    statistics=statistics_gen.outputs['statistics'],
    schema=schema_gen.outputs['schema']
)

context.run(example_validator)

ExecutionResult(
    component_id: ExampleValidator
    execution_id: 4
    outputs:
        anomalies: OutputChannel(artifact_type=ExampleAnomalies, producer_component_id=ExampleValidator, output_key=anomalies, additional_properties={}, additional_custom_properties={}, _input_trigger=None)

In [10]:
context.show(example_validator.outputs['anomalies'])

In [11]:
transform = Transform(
    examples=example_gen.outputs['examples'],
    schema=schema_gen.outputs['schema'],
    module_file=os.path.abspath(os.path.join('modules', 'transform.py')),
)

context.run(transform)

running bdist_wheel
running build
running build_py
creating build
creating build/lib
copying trainer.py -> build/lib
copying transform.py -> build/lib
copying tuner.py -> build/lib
installing to /tmp/tmp78zvin4v
running install
running install_lib
copying build/lib/tuner.py -> /tmp/tmp78zvin4v
copying build/lib/trainer.py -> /tmp/tmp78zvin4v
copying build/lib/transform.py -> /tmp/tmp78zvin4v
running install_egg_info


/media/mausneg/ntfs_disk/Surya/Language Programming/Python/Customer-Churn-Classification-MLOps/.venv/lib/python3.10/site-packages/setuptools/command/install.py:34: SetuptoolsDeprecationWarning: setup.py install is deprecated. Use build and pip and other standards-based tools.
  warnings.warn(


running egg_info
creating tfx_user_code_Transform.egg-info
writing tfx_user_code_Transform.egg-info/PKG-INFO
writing dependency_links to tfx_user_code_Transform.egg-info/dependency_links.txt
writing top-level names to tfx_user_code_Transform.egg-info/top_level.txt
writing manifest file 'tfx_user_code_Transform.egg-info/SOURCES.txt'
reading manifest file 'tfx_user_code_Transform.egg-info/SOURCES.txt'
writing manifest file 'tfx_user_code_Transform.egg-info/SOURCES.txt'
Copying tfx_user_code_Transform.egg-info to /tmp/tmp78zvin4v/tfx_user_code_Transform-0.0+ebab2a791ecfe0105c495fb6b07f260b41fc50a2df4c2ae677afa61168022cca-py3.10.egg-info
running install_scripts
creating /tmp/tmp78zvin4v/tfx_user_code_Transform-0.0+ebab2a791ecfe0105c495fb6b07f260b41fc50a2df4c2ae677afa61168022cca.dist-info/WHEEL
creating '/tmp/tmpg7h3ri0t/tfx_user_code_Transform-0.0+ebab2a791ecfe0105c495fb6b07f260b41fc50a2df4c2ae677afa61168022cca-py3-none-any.whl' and adding '/tmp/tmp78zvin4v' to it
adding 'trainer.py'
addin


[notice] A new release of pip is available: 23.0.1 -> 24.3.1
[notice] To update, run: pip install --upgrade pip


Processing ./pipelines/churn-pipeline/_wheels/tfx_user_code_Transform-0.0+ebab2a791ecfe0105c495fb6b07f260b41fc50a2df4c2ae677afa61168022cca-py3-none-any.whl


...

INFO:tensorflow:tensorflow_decision_forests is not available.


INFO:tensorflow:tensorflow_decision_forests is not available.


INFO:tensorflow:tensorflow_text is not available.


INFO:tensorflow:tensorflow_text is not available.


ExecutionResult(
    component_id: Transform
    execution_id: 5
    outputs:
        transform_graph: OutputChannel(artifact_type=TransformGraph, producer_component_id=Transform, output_key=transform_graph, additional_properties={}, additional_custom_properties={}, _input_trigger=None
        transformed_examples: OutputChannel(artifact_type=Examples, producer_component_id=Transform, output_key=transformed_examples, additional_properties={}, additional_custom_properties={}, _input_trigger=None
        updated_analyzer_cache: OutputChannel(artifact_type=TransformCache, producer_component_id=Transform, output_key=updated_analyzer_cache, additional_properties={}, additional_custom_properties={}, _input_trigger=None
        pre_transform_schema: OutputChannel(artifact_type=Schema, producer_component_id=Transform, output_key=pre_transform_schema, additional_properties={}, additional_custom_properties={}, _input_trigger=None
        pre_transform_stats: OutputChannel(artifact_type=ExampleStatistics, producer_component_id=Transform, output_key=pre_transform_stats, additional_properties={}, additional_custom_properties={}, _input_trigger=None
        post_transform_schema: OutputChannel(artifact_type=Schema, producer_component_id=Transform, output_key=post_transform_schema, additional_properties={}, additional_custom_properties={}, _input_trigger=None
        post_transform_stats: OutputChannel(artifact_type=ExampleStatistics, producer_component_id=Transform, output_key=post_transform_stats, additional_properties={}, additional_custom_properties={}, _input_trigger=None
        post_transform_anomalies: OutputChannel(artifact_type=ExampleAnomalies, producer_component_id=Transform, output_key=post_transform_anomalies, additional_properties={}, additional_custom_properties={}, _input_trigger=None)

In [14]:
tuner = Tuner(
    module_file=os.path.abspath(os.path.join('modules', 'tuner.py')),
    examples=transform.outputs['transformed_examples'],
    transform_graph=transform.outputs['transform_graph'],   
    schema=schema_gen.outputs['schema'],
    train_args=trainer_pb2.TrainArgs(splits=['train'], num_steps=8000),
    eval_args=trainer_pb2.EvalArgs(splits=['eval'], num_steps=6000),
)

context.run(tuner)

Trial 10 Complete [00h 01m 34s]
val_binary_accuracy: 0.5994244813919067

Best val_binary_accuracy So Far: 0.6228229403495789
Total elapsed time: 00h 15m 15s
Results summary
Results in pipelines/churn-pipeline/.temp/8/churn
Showing 10 best trials
Objective(name="val_binary_accuracy", direction="max")

Trial 08 summary
Hyperparameters:
num_layers: 2
units_0: 384
dropout_0: 0.1
learning_rate: 0.001
units_1: 352
dropout_1: 0.30000000000000004
units_2: 256
dropout_2: 0.1
tuner/epochs: 2
tuner/initial_epoch: 0
tuner/bracket: 3
tuner/round: 0
Score: 0.6228229403495789

Trial 09 summary
Hyperparameters:
num_layers: 2
units_0: 480
dropout_0: 0.4
learning_rate: 0.001
units_1: 192
dropout_1: 0.1
units_2: 96
dropout_2: 0.4
tuner/epochs: 2
tuner/initial_epoch: 0
tuner/bracket: 3
tuner/round: 0
Score: 0.5994244813919067

Trial 03 summary
Hyperparameters:
num_layers: 3
units_0: 384
dropout_0: 0.5
learning_rate: 0.01
units_1: 224
dropout_1: 0.1
units_2: 64
dropout_2: 0.1
tuner/epochs: 2
tuner/initial_

ExecutionResult(
    component_id: Tuner
    execution_id: 8
    outputs:
        best_hyperparameters: OutputChannel(artifact_type=HyperParameters, producer_component_id=Tuner, output_key=best_hyperparameters, additional_properties={}, additional_custom_properties={}, _input_trigger=None
        tuner_results: OutputChannel(artifact_type=TunerResults, producer_component_id=Tuner, output_key=tuner_results, additional_properties={}, additional_custom_properties={}, _input_trigger=None)

In [15]:
trainer = Trainer(
    module_file=os.path.abspath(os.path.join('modules', 'trainer.py')),
    examples=transform.outputs['transformed_examples'],
    transform_graph=transform.outputs['transform_graph'],
    schema=schema_gen.outputs['schema'],
    hyperparameters=tuner.outputs['best_hyperparameters'],
    train_args=trainer_pb2.TrainArgs(splits=['train'], num_steps=8000),
    eval_args=trainer_pb2.EvalArgs(splits=['eval'], num_steps=6000)
)

context.run(trainer)

running bdist_wheel
running build
running build_py
creating build
creating build/lib
copying trainer.py -> build/lib
copying transform.py -> build/lib
copying tuner.py -> build/lib
installing to /tmp/tmp9ga7pkej
running install
running install_lib
copying build/lib/tuner.py -> /tmp/tmp9ga7pkej
copying build/lib/trainer.py -> /tmp/tmp9ga7pkej
copying build/lib/transform.py -> /tmp/tmp9ga7pkej
running install_egg_info
running egg_info
creating tfx_user_code_Trainer.egg-info
writing tfx_user_code_Trainer.egg-info/PKG-INFO
writing dependency_links to tfx_user_code_Trainer.egg-info/dependency_links.txt
writing top-level names to tfx_user_code_Trainer.egg-info/top_level.txt
writing manifest file 'tfx_user_code_Trainer.egg-info/SOURCES.txt'
reading manifest file 'tfx_user_code_Trainer.egg-info/SOURCES.txt'
writing manifest file 'tfx_user_code_Trainer.egg-info/SOURCES.txt'
Copying tfx_user_code_Trainer.egg-info to /tmp/tmp9ga7pkej/tfx_user_code_Trainer-0.0+ebab2a791ecfe0105c495fb6b07f260b41fc5

/media/mausneg/ntfs_disk/Surya/Language Programming/Python/Customer-Churn-Classification-MLOps/.venv/lib/python3.10/site-packages/setuptools/command/install.py:34: SetuptoolsDeprecationWarning: setup.py install is deprecated. Use build and pip and other standards-based tools.
  warnings.warn(


Processing ./pipelines/churn-pipeline/_wheels/tfx_user_code_Trainer-0.0+ebab2a791ecfe0105c495fb6b07f260b41fc50a2df4c2ae677afa61168022cca-py3-none-any.whl



[notice] A new release of pip is available: 23.0.1 -> 24.3.1
[notice] To update, run: pip install --upgrade pip


Epoch 1/2
8000/8000 [==============================] - 61s 7ms/step - loss: 0.0695 - binary_accuracy: 0.9810 - val_loss: 9.6387 - val_binary_accuracy: 0.4714
Epoch 2/2
8000/8000 [==============================] - 55s 7ms/step - loss: 0.0918 - binary_accuracy: 0.9768 - val_loss: 1.0216 - val_binary_accuracy: 0.5892
INFO:tensorflow:struct2tensor is not available.


...

INFO:tensorflow:tensorflow_text is not available.


INFO:tensorflow:tensorflow_text is not available.


INFO:tensorflow:Assets written to: pipelines/churn-pipeline/Trainer/model/9/Format-Serving/assets


INFO:tensorflow:Assets written to: pipelines/churn-pipeline/Trainer/model/9/Format-Serving/assets


ExecutionResult(
    component_id: Trainer
    execution_id: 9
    outputs:
        model: OutputChannel(artifact_type=Model, producer_component_id=Trainer, output_key=model, additional_properties={}, additional_custom_properties={}, _input_trigger=None
        model_run: OutputChannel(artifact_type=ModelRun, producer_component_id=Trainer, output_key=model_run, additional_properties={}, additional_custom_properties={}, _input_trigger=None)

In [16]:
model_resolver = Resolver(
    strategy_class= LatestBlessedModelStrategy,
    model = Channel(type=Model),
    model_blessing = Channel(type=ModelBlessing)
).with_id('Latest_blessed_model_resolver')
 
context.run(model_resolver)

ExecutionResult(
    component_id: Latest_blessed_model_resolver
    execution_id: 10
    outputs:
        model: OutputChannel(artifact_type=Model, producer_component_id=Latest_blessed_model_resolver, output_key=model, additional_properties={}, additional_custom_properties={}, _input_trigger=None
        model_blessing: OutputChannel(artifact_type=ModelBlessing, producer_component_id=Latest_blessed_model_resolver, output_key=model_blessing, additional_properties={}, additional_custom_properties={}, _input_trigger=None)

In [17]:
eval_config = tfma.EvalConfig(
    model_specs=[tfma.ModelSpec(label_key='Churn')],
    slicing_specs=[tfma.SlicingSpec()],
    metrics_specs=[
        tfma.MetricsSpec(metrics=[
            tfma.MetricConfig(class_name='ExampleCount'),
            tfma.MetricConfig(class_name='AUC'),
            tfma.MetricConfig(class_name='FalsePositives'),
            tfma.MetricConfig(class_name='TruePositives'),
            tfma.MetricConfig(class_name='FalseNegatives'),
            tfma.MetricConfig(class_name='TrueNegatives'),
            tfma.MetricConfig(class_name='BinaryAccuracy',
                threshold=tfma.MetricThreshold(
                    value_threshold=tfma.GenericValueThreshold(
                        lower_bound={'value':0.5}),
                    change_threshold=tfma.GenericChangeThreshold(
                        direction=tfma.MetricDirection.HIGHER_IS_BETTER,
                        absolute={'value':0.0001})
                    )
            )
        ])
    ]
)

In [18]:
evaluator = Evaluator(
    examples=example_gen.outputs['examples'],
    model=trainer.outputs['model'],
    baseline_model=model_resolver.outputs['model'],
    eval_config=eval_config)
 
context.run(evaluator)

Instructions for updating:
Use eager execution and: 
`tf.data.TFRecordDataset(path)`


Instructions for updating:
Use eager execution and: 
`tf.data.TFRecordDataset(path)`


ExecutionResult(
    component_id: Evaluator
    execution_id: 11
    outputs:
        evaluation: OutputChannel(artifact_type=ModelEvaluation, producer_component_id=Evaluator, output_key=evaluation, additional_properties={}, additional_custom_properties={}, _input_trigger=None
        blessing: OutputChannel(artifact_type=ModelBlessing, producer_component_id=Evaluator, output_key=blessing, additional_properties={}, additional_custom_properties={}, _input_trigger=None)

In [19]:
eval_result = evaluator.outputs['evaluation'].get()[0].uri
tfma_result = tfma.load_eval_result(eval_result)
tfma.view.render_slicing_metrics(tfma_result)
tfma.addons.fairness.view.widget_view.render_fairness_indicator(
    tfma_result
)

FairnessIndicatorViewer(slicingMetrics=[{'sliceValue': 'Overall', 'slice': 'Overall', 'metrics': {'binary_accu…

In [20]:
pusher = Pusher(
    model=trainer.outputs['model'],
    model_blessing=evaluator.outputs['blessing'],
    push_destination=pusher_pb2.PushDestination(
        filesystem=pusher_pb2.PushDestination.Filesystem(
            base_directory='serving_model_dir/customer-churn-model'
        )
    )
)
context.run(pusher)

ExecutionResult(
    component_id: Pusher
    execution_id: 12
    outputs:
        pushed_model: OutputChannel(artifact_type=PushedModel, producer_component_id=Pusher, output_key=pushed_model, additional_properties={}, additional_custom_properties={}, _input_trigger=None)

In [21]:
context.show(pusher.outputs['pushed_model'])

In [22]:
!docker build -t mausneg/customer-churn-tf-serving .

[+] Building 0.0s (0/1)                                          docker:default
[+] Building 0.2s (1/2)                                          docker:default
 => [internal] load build definition from Dockerfile                       0.0s
 => => transferring dockerfile: 138B                                       0.0s
 => [internal] load metadata for docker.io/tensorflow/serving:latest       0.2s
[+] Building 0.3s (1/2)                                          docker:default
 => [internal] load build definition from Dockerfile                       0.0s
 => => transferring dockerfile: 138B                                       0.0s
 => [internal] load metadata for docker.io/tensorflow/serving:latest       0.3s
[+] Building 0.5s (1/2)                                          docker:default
 => [internal] load build definition from Dockerfile                       0.0s
 => => transferring dockerfile: 138B                                       0.0s
 => [internal] load metadata for docker.

In [23]:
!docker run -p 8080:8501 -d mausneg/customer-churn-tf-serving

f277dd2ce658a6af7bfea53c95b95bc2b0a92a9e0cd6335e50b4cd61c5d95266


In [25]:
import requests
from pprint import PrettyPrinter
 
pp = PrettyPrinter()
pp.pprint(requests.get("http://localhost:8080/v1/models/customer-churn-model").json())

{'model_version_status': [{'state': 'AVAILABLE',
                           'status': {'error_code': 'OK', 'error_message': ''},
                           'version': '1730771405'}]}


In [26]:
!docker stop $(docker ps -a -q)

f277dd2ce658
3ba4792bfe49
cd49f25c8695
c7c054caa76f
fc62988f0382
8253c7714747
90fc6302cd95
6c37e75dc4f0
22876a521486
